In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## 1. Análisis de Procedimientos a Nivel País

En este análisis se quieren responder las siguiente preguntas:

1. ¿Cómo es el ranking de los hospitales para cada uno de los procedimientos realizads?
2. ¿Quién es el Hospital que realiza más de X procedimientos?
3. ¿El procedimiento X es realizado en todo el país? ¿En qué partes en específico se realiza?

Para responder esta pregunta se utilizará la base de datos de Egresos de Hospitales adscritos
a GRD entre 2019 y 2021.


In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
cie_9 = pd.read_excel("../data/external/CIE-9.xlsx", dtype={"Código": str})
cie_10 = pd.read_excel("../data/external/CIE-10.xlsx")
esquema_hospitales = pd.read_excel(
    "../data/external/Esquema_Registro-2023.xls", sheet_name=4, header=7
).dropna(how="all", axis=1)

In [5]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

In [6]:
display(df_procesada.head(10))

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA,RANGO_ETARIO,cod_comuna,codregion,Nombre Región
0,118100,1314867.0,MUJER,1988-06-19,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA A,CONCEPCIÓN,...,1.0,NaN,5.0,2019.0,3.0,2019-3,"(30, 40]",8108,8,del Biobío
1,118100,1354418.0,MUJER,1988-03-27,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA D,CONCEPCIÓN,...,1.0,NaN,4.0,2019.0,3.0,2019-3,"(30, 40]",8108,8,del Biobío
2,118100,239861.0,MUJER,2017-06-17,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA B,CONCEPCIÓN,...,2.0,NaN,6.0,2019.0,3.0,2019-3,"(0, 10]",8101,8,del Biobío
3,118100,330326.0,HOMBRE,2017-12-04,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA A,CONCEPCIÓN,...,2.0,NaN,2.0,2019.0,3.0,2019-3,"(0, 10]",8101,8,del Biobío
4,118100,1369293.0,MUJER,1995-12-06,OTRO,CONCEPCION,FLORIDA,CHILE,FONASA C,CONCEPCIÓN,...,1.0,NaN,6.0,2019.0,3.0,2019-3,"(20, 30]",8104,8,del Biobío
5,118100,469385.0,MUJER,1942-06-29,OTRO,CONCEPCION,CHIGUAYANTE,CHILE,FONASA B,CONCEPCIÓN,...,3.0,NaN,47.0,2019.0,3.0,2019-3,"(70, 80]",8103,8,del Biobío
6,118100,1146635.0,MUJER,1951-07-14,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA B,CONCEPCIÓN,...,3.0,NaN,43.0,2019.0,3.0,2019-3,"(60, 70]",8101,8,del Biobío
7,118100,93557.0,HOMBRE,1938-03-04,OTRO,CONCEPCION,LOTA,CHILE,FONASA D,CONCEPCIÓN,...,1.0,NaN,41.0,2019.0,3.0,2019-3,"(80, 90]",8106,8,del Biobío
8,118100,674943.0,MUJER,1962-08-21,OTRO,CONCEPCION,SANTA JUANA,CHILE,FONASA B,CONCEPCIÓN,...,2.0,HOSPITAL CLORINDA AVELLO (SANTA JUANA),33.0,2019.0,3.0,2019-3,"(50, 60]",8109,8,del Biobío
9,118100,1079955.0,HOMBRE,1944-12-07,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA B,CONCEPCIÓN,...,3.0,NaN,31.0,2019.0,3.0,2019-3,"(70, 80]",8101,8,del Biobío


Los procedimientos en esta base de datos se encuentran en las columnas "PROCEDIMIENTOSX", donde X
va desde 1 hasta 30. Para analizar esta base de forma más expedita, se transformará de formato
ancho a formato largo, utilizando como identificador a las columnas ["COD_HOSPITAL", "ANIO_EGRESO",
"CIP_ENCRIPTADO", "DIAGNOSTICO1"]. Cabe destacar que la columna "CIP_ENCRIPTADO" hace referencia al
RUT del paciente.


In [7]:
procedimientos_formato_long = pd.melt(
    df_procesada,
    id_vars=[
        "COD_HOSPITAL",
        "ANIO_EGRESO",
        "TIPO_ACTIVIDAD",
        "CIP_ENCRIPTADO",
        "TIPOALTA",
        "DIAGNOSTICO1",
        "IR_29301_PESO",
        "IR_29301_SEVERIDAD",
    ],
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [8]:
var_categorica_a_contar = "procedimiento"
desglose_a_obtener = ["ANIO_EGRESO", "COD_HOSPITAL"]
grupo_ranking = ["ANIO_EGRESO", "procedimiento"]

conteo_y_ranking_procedimientos = (
    build_features.calcular_conteo_y_ranking_sobre_variable_categorica(
        procedimientos_formato_long,
        var_categorica_a_contar,
        desglose_a_obtener,
        grupo_ranking,
    )
)

In [9]:
display(conteo_y_ranking_procedimientos.head(20))

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento
180164,2021.0,113180,99.99,4,1,2,18,0.222222
160709,2021.0,108100,99.99,2,2,1,18,0.111111
148422,2021.0,103100,99.99,1,3,0,18,0.055556
152362,2021.0,105100,99.99,1,4,0,18,0.055556
166180,2021.0,110100,99.99,1,5,0,18,0.055556
167426,2021.0,110120,99.99,1,6,0,18,0.055556
179347,2021.0,113130,99.99,1,7,0,18,0.055556
182873,2021.0,114101,99.99,1,8,0,18,0.055556
187939,2021.0,115107,99.99,1,9,0,18,0.055556
188727,2021.0,115110,99.99,1,10,0,18,0.055556


El resultado muestra el ranking para todos los procedimientos asignados en el país en cada uno de
los años analizados (2019 a 2021).

A modo de ejemplo, se ilustra el ranking en el año 2021 para el procedimiento 99.99. En este caso,
se observa que los hospitales con código 108100 y 113180 están en los primeros lugares, realizando
2 procedimientos cada uno.

Ahora, si se filtra por el hospital del tórax, y sólo los procedimientos donde el hospital sea
ranking 1, entonces se obtendrán procedimientos altamente relevantes para el país.


In [10]:
conteo_y_ranking_procedimientos = conteo_y_ranking_procedimientos.merge(
    cie_9, how="left", left_on="procedimiento", right_on="Código"
).merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)


In [11]:
display(conteo_y_ranking_procedimientos)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento,Código,Descripción,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
0,2021.0,113180,99.99,4,1,2,18,0.222222,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,113180,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital El Pino (Santiago, San Bernardo)",13401,13
1,2021.0,108100,99.99,2,2,1,18,0.111111,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,108100,Servicio de Salud Aconcagua,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Camilo de San Felipe,5701,5
2,2021.0,103100,99.99,1,3,0,18,0.055556,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,103100,Servicio de Salud Antofagasta,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Dr. Leonardo Guzmán (Antofagasta),2101,2
3,2021.0,105100,99.99,1,4,0,18,0.055556,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,105100,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Juan de Dios (La Serena),4101,4
4,2021.0,110100,99.99,1,5,0,18,0.055556,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,110100,Servicio de Salud Metropolitano Occidente,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital San Juan de Dios (Santiago, Santiago)",13101,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214719,2019.0,105101,00.01,65,1,52,80,0.812500,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,105101,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Pablo (Coquimbo),4102,4
214720,2019.0,106100,00.01,13,2,12,80,0.162500,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,106100,Servicio de Salud Valparaíso San Antonio,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Carlos Van Buren (Valparaíso),5101,5
214721,2019.0,113100,00.01,1,3,0,80,0.012500,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,113100,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital Barros Luco Trudeau (Santiago, San Mi...",13130,13
214722,2019.0,120101,00.01,1,4,-1,80,0.012500,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,120101,Servicio de Salud Biobío,Perteneciente,1,Hospital (Alta Complejidad),4.0,Complejo Asistencial Dr. Víctor Ríos Ruiz (Los...,8301,8


Ahora se puede observar el nombre del procedimiento analizado y el nombre del hospital visto.


In [12]:
conteo_procedimientos_torax = conteo_y_ranking_procedimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX"
).sort_values(["ANIO_EGRESO", "conteo_procedimiento"], ascending=False)

mejores_procedimientos_torax = conteo_procedimientos_torax.query(
    "ranking_conteo_procedimiento == 1"
)

display(mejores_procedimientos_torax)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento,Código,Descripción,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
54292,2021.0,112103,39.61,626,1,68,3184,0.196608,39.61,CIRCULACION EXTRACORPOREA PARA CIRUGIA CORAZON...,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
54248,2021.0,112103,39.64,584,1,246,2020,0.289109,39.64,MARCAPASOS CORAZON INTRAOPERATORIO,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
52930,2021.0,112103,42.23,391,1,342,607,0.644152,42.23,ESOFAGOSCOPIA OTRA,...,09. (42 54) OPERACIONES SOBRE EL APARATO DIGES...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
57290,2021.0,112103,37.22,280,1,22,801,0.349563,37.22,CATETERISMO CORAZON LADO IZQUIERDO,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
13241,2021.0,112103,89.46,278,1,82,1355,0.205166,89.46,COMPROBACION FORMA ONDA ARTEFACTO MARCAPASOS C...,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200772,2019.0,112103,34.89,2,1,1,6,0.333333,34.89,OPERACION DIAFRAGMA.OTRA,...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
201686,2019.0,112103,33.49,2,1,1,10,0.200000,33.49,OPERACION PULMON REPARADORA Y PLASTICA OTRA,...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
210994,2019.0,112103,07.81,2,1,1,10,0.200000,07.81,ESCISION TIMO PARCIAL OTRA,...,02. (06 07) OPERACIONES SOBRE EL SISTEMA ENDOC...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
198511,2019.0,112103,38.46,1,1,0,3,0.333333,38.46,RESECCION ARTERIA ABDOMEN CON SUSTITUCION,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13


Los resultados muestran que el tórax en el año 2021 fue altamente relevante en los procedimientos
39.61, 39.64, 42.23, entre otros. Además, se puede observar la diferencia que tiene el Tórax
con los siguientes establecimientos de salud y el total de procedimientos realizados en el país.


Ahora, se quiere saber la cantidad de procedimientos, pero sabiendo si fue realizado de forma
ambulatoria, hospitalizada, cirugia mayor ambulatoria, etc...


In [13]:
var_categorica_a_contar = "procedimiento"
desglose_a_obtener = ["ANIO_EGRESO", "COD_HOSPITAL", "TIPO_ACTIVIDAD"]
grupo_ranking = ["ANIO_EGRESO", "TIPO_ACTIVIDAD", "procedimiento"]

conteo_y_ranking_procedimientos_por_tipoalta = (
    build_features.calcular_conteo_y_ranking_sobre_variable_categorica(
        procedimientos_formato_long,
        var_categorica_a_contar,
        desglose_a_obtener,
        grupo_ranking,
    )
)

In [14]:
conteo_y_ranking_procedimientos_por_tipoalta = conteo_y_ranking_procedimientos_por_tipoalta.merge(
    cie_9, how="left", left_on="procedimiento", right_on="Código"
).merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)


In [15]:
with pd.ExcelWriter("../data/interim/procedimientos_grd.xlsx") as file:
    conteo_procedimientos_torax.to_excel(file, sheet_name="procedimientos_torax", index=False)
    conteo_y_ranking_procedimientos.to_excel(file, sheet_name="resumen_nacional", index=False)
    conteo_y_ranking_procedimientos_por_tipoalta.to_excel(
        file, sheet_name="resumen_nacional_tipoalta", index=False
    )

## 2. Obtención de perfiles de pacientes por procedimientos

En este apartado se quieren responder las siguientes preguntas:

1. ¿Cuál es la **mortalidad** de cada uno de los procedimientos?
2. ¿Cuál es el **peso GRD** promedio de cada uno de los procedimientos?
3. ¿Cuál es la **severidad** de cada uno de los procedimientos?
4. Dentro de los pacientes muertos, **¿Cuál es la severidad que tienen?**

Ahora, la base de GRD tiene hasta 30 procedimientos, por lo que en un mismo egreso se pueden
repetir procedimientos. Para reducir la complejidad del análisis, se utilizará solamente el primer
procedimiento para agrupar por severidad y mortalidad. Esto, ya que 1 procedimiento corresponderá
a 1 egreso.

Por lo tanto, la base de datos se debe mantener como está, pero solamente seleccionando las variables
recién mencionadas


In [16]:
display(procedimientos_formato_long)

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,CIP_ENCRIPTADO,TIPOALTA,DIAGNOSTICO1,IR_29301_PESO,IR_29301_SEVERIDAD,variable,procedimiento
0,118100,2019.0,HOSPITALIZACIÓN,1314867.0,DOMICILIO,O82.0,0.5744,2.0,PROCEDIMIENTO1,74.1
1,118100,2019.0,HOSPITALIZACIÓN,1354418.0,DOMICILIO,O26.9,0.2951,1.0,PROCEDIMIENTO1,88.78
2,118100,2019.0,HOSPITALIZACIÓN,239861.0,DOMICILIO,K92.0,0.6736,2.0,PROCEDIMIENTO1,87.03
3,118100,2019.0,HOSPITALIZACIÓN,330326.0,DOMICILIO,A08.3,0.5475,2.0,PROCEDIMIENTO1,90.92
4,118100,2019.0,HOSPITALIZACIÓN,1369293.0,DOMICILIO,O60.0,0.3107,1.0,PROCEDIMIENTO1,75.34
...,...,...,...,...,...,...,...,...,...,...
82471585,105100,2021.0,HOSPITALIZACIÓN,68882797.0,DOMICILIO,N39.0,1.6547,3.0,PROCEDIMIENTO30,NaN
82471586,120101,2021.0,HOSPITALIZACIÓN,97789022.0,DOMICILIO,K40.9,0.8122,1.0,PROCEDIMIENTO30,NaN
82471587,114101,2021.0,HOSPITALIZACIÓN,71164179.0,DOMICILIO,Z51.8,1.0867,2.0,PROCEDIMIENTO30,NaN
82471588,107100,2021.0,HOSPITALIZACIÓN,78884679.0,DOMICILIO,K80.0,1.1472,2.0,PROCEDIMIENTO30,NaN


Ahora que se tiene la base de datos con menos variables, se quiere responder a la pregunta:

¿Cuál es la mortalidad de cada uno de los procedimientos?


In [17]:
desglose_procedimientos = procedimientos_formato_long.groupby(
    [
        "COD_HOSPITAL",
        "ANIO_EGRESO",
        "TIPO_ACTIVIDAD",
        "TIPOALTA",
        "procedimiento",
        "IR_29301_SEVERIDAD",
    ]
).agg(cantidad_procedimientos=("procedimiento", "count"), peso_grd_medio=("IR_29301_PESO", "mean"))

display(desglose_procedimientos.head(10))

desglose_procedimientos = desglose_procedimientos.reset_index().sort_values(
    ["COD_HOSPITAL", "ANIO_EGRESO", "cantidad_procedimientos"]
)

cantidad_procedimientos  \
COD_HOSPITAL ANIO_EGRESO TIPO_ACTIVIDAD                  TIPOALTA  procedimiento IR_29301_SEVERIDAD                            
101100       2019.0      CIRUGÍA MAYOR AMBULATORIA (CMA) DOMICILIO 03.92         0.0                                      44   
                                                                   04.07         0.0                                       1   
                                                                   04.11         0.0                                       1   
                                                                   04.43         0.0                                      45   
                                                                   04.81         0.0                                      15   
                                                                   06.31         0.0                                       1   
                                                                   06.7          0.0                                       1   
                                                                   08.01         0.0                                       1   
                                                                   08.20         0.0                                       6   
                                                                   08.22         0.0                                       1   

                                                                                                     peso_grd_medio  
COD_HOSPITAL ANIO_EGRESO TIPO_ACTIVIDAD                  TIPOALTA  procedimiento IR_29301_SEVERIDAD                  
101100       2019.0      CIRUGÍA MAYOR AMBULATORIA (CMA) DOMICILIO 03.92         0.0                       0.148200  
                                                                   04.07         0.0                       1.216700  
                                                                   04.11         0.0                       0.297300  
                                                                   04.43         0.0                       0.304500  
                                                                   04.81         0.0                       0.147913  
                                                                   06.31         0.0                       0.987000  
                                                                   06.7          0.0                       0.552400  
                                                                   08.01         0.0                       0.188500  
                                                                   08.20         0.0                       0.329967  
                                                                   08.22         0.0                       0.273700

Los resultados anteriores indican lo siguiente:

El hospital 101100 en el año 2019 tuvo 44 procedimientos 1 03.92 que tuvieron una severidad de 0.
Estos procedimientos fueron realizados en una cirugía mayor ambulatorio (CMA), y la alta de
sus pacientes fueron en DOMICILIO. Dentro de los 44 egresos, estos tuvieron una complejidad
promedio de 0.148200.

Ahora, para obtener la mortalidad de cada procedimiento es necesario consultar los pacientes
que tienen un TIPOALTA igual a FALLECIDO.


In [18]:
display(desglose_procedimientos.query("TIPOALTA == 'FALLECIDO' and procedimiento == '00.66'"))

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,TIPOALTA,procedimiento,IR_29301_SEVERIDAD,cantidad_procedimientos,peso_grd_medio
3496,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,2.0,2,1.818300
3497,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,3,2.484000
8207,101100,2020.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,3,3.048567
12628,101100,2021.0,HOSPITALIZACIÓN,FALLECIDO,00.66,1.0,2,1.718400
12629,101100,2021.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,7,2.725957
...,...,...,...,...,...,...,...,...
784235,126100,2020.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,4,3.741600
788411,126100,2021.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,5,4.995440
797134,129100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,1,2.484000
799585,129100,2020.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,2,2.112300


Los resultados anteriores indican que en el Hospital 101100 en el año 2021, 7 de los procedimientos
00.66 tuvieron fallecimientos en una hospitalización. Estos procedimientos tenían una severidad de 3,
con un peso GRD promedio de 2.7260.

A modo de ejemplo, se realizará la consulta en la base de datos y se mostrarán los pacientes
que cumplen los criterios.


In [19]:
consulta_inicial = df_procesada.query(
    "ANIO_EGRESO == 2021.0 and COD_HOSPITAL == 101100 and "
    "IR_29301_SEVERIDAD == 3.0 and TIPOALTA == 'FALLECIDO'"
)

filas_con_procedimiento = (
    consulta_inicial[vars_procedimientos.keys()]
    .apply(lambda x: x.str.contains("00.66"))
    .sum(axis=1)
    .astype(bool)
)

display(consulta_inicial[filas_con_procedimiento])

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA,RANGO_ETARIO,cod_comuna,codregion,Nombre Región
2101790,101100,78353743.0,HOMBRE,1949-06-20,NINGUNO,ARICA,ARICA,CHILE,FONASA B,ARICA,...,3.0,NaN,2.0,2021.0,4.0,2021-4,"(70, 80]",15101,15,de Arica y Parinacota
2495416,101100,70827726.0,MUJER,1946-04-20,NINGUNO,ARICA,ARICA,CHILE,FONASA C,ARICA,...,3.0,NaN,12.0,2021.0,10.0,2021-10,"(70, 80]",15101,15,de Arica y Parinacota
2513407,101100,70124532.0,MUJER,1954-09-08,NINGUNO,ARICA,ARICA,CHILE,FONASA A,ARICA,...,3.0,NaN,1.0,2021.0,2.0,2021-2,"(60, 70]",15101,15,de Arica y Parinacota
2536108,101100,68419768.0,HOMBRE,1957-01-26,NINGUNO,ARICA,ARICA,CHILE,FONASA B,ARICA,...,3.0,NaN,5.0,2021.0,6.0,2021-6,"(60, 70]",15101,15,de Arica y Parinacota
2592167,101100,70259268.0,MUJER,1945-10-18,NINGUNO,ARICA,ARICA,CHILE,FONASA B,ARICA,...,3.0,NaN,2.0,2021.0,11.0,2021-11,"(70, 80]",15101,15,de Arica y Parinacota
2675921,101100,71418919.0,MUJER,1970-02-17,NINGUNO,ARICA,ARICA,CHILE,FONASA B,ARICA,...,3.0,NaN,10.0,2021.0,12.0,2021-12,"(50, 60]",15101,15,de Arica y Parinacota
2708809,101100,68127172.0,HOMBRE,1939-08-01,NINGUNO,ARICA,ARICA,CHILE,FONASA C,ARICA,...,3.0,NaN,3.0,2021.0,4.0,2021-4,"(80, 90]",15101,15,de Arica y Parinacota


Sin embargo, es necesario saber cuánto es el porcentaje de muertos del total.


In [20]:
total_proceds = desglose_procedimientos.groupby(["COD_HOSPITAL", "ANIO_EGRESO", "procedimiento"])[
    "cantidad_procedimientos"
].transform("sum")

desglose_procedimientos["total_de_procedimientos"] = total_proceds

desglose_procedimientos["ratio_de_procedimientos"] = round(
    desglose_procedimientos["cantidad_procedimientos"]
    / desglose_procedimientos["total_de_procedimientos"],
    2,
)

In [21]:
desglose_procedimientos.query("COD_HOSPITAL == 101100 and ANIO_EGRESO == 2019.0 "
                              "and procedimiento == '00.66'")

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,TIPOALTA,procedimiento,IR_29301_SEVERIDAD,cantidad_procedimientos,peso_grd_medio,total_de_procedimientos,ratio_de_procedimientos
1234,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,3.0,2,2.4840,34,0.06
3496,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,2.0,2,1.8183,34,0.06
3497,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,3,2.4840,34,0.09
1233,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,2.0,12,1.8183,34,0.35
1232,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,1.0,15,1.7184,34,0.44


Los resultados anteriores se pueden interpretar de la siguiente manera:

- El hospital 101100, en el año 2019 tuvo 5 procedimientos en donde
  la persona falleció realizandose un procedimiento 00.66. Estos egresos tenían una severidad 2
  y 3, y corresponden al 15% de todos los procedimientos realizados.


In [22]:
desglose_procedimientos = desglose_procedimientos.merge(
    cie_9, how="left", left_on="procedimiento", right_on="Código"
).merge(
    esquema_hospitales,
    how="left",
    left_on="COD_HOSPITAL",
    right_on="Código nuevo Establecimiento",
)

In [23]:
desglose_procedimientos.to_excel(
    "../data/interim/mortalidad_y_severidad_procedimientos.xlsx",
    index=False,
)


En el Hospital del Tórax existe un interés por saber cuál es la condición de ingreso de los
pacientes que se hicieron una angioplastia el 2019 y que fallecieron dentro de una hospitalizacion
diurna.

In [24]:
casos_angio = df_procesada.query(
    "ANIO_EGRESO == 2019.0 and COD_HOSPITAL == @HOSPITAL_DEL_TORAX and TIPOALTA == 'FALLECIDO' "
    "and TIPO_ACTIVIDAD == 'HOSPITALIZACIÓN DIURNA'"
)

filas_con_procedimiento = (
    casos_angio[vars_procedimientos.keys()]
    .apply(lambda x: x.str.contains("00.66"))
    .sum(axis=1)
    .astype(bool)
)

muertos_torax_angio = casos_angio[filas_con_procedimiento]

In [25]:
display(muertos_torax_angio[
    [
        "SEXO",
        "NACIONALIDAD",
        "PREVISION",
        "TIPO_PROCEDENCIA",
        "TIPO_INGRESO",
        "IR_29301_PESO",
        "IR_29301_SEVERIDAD",
        "HOSPPROCEDENCIA",
        "Nombre Región",
        "COMUNA",
        "RANGO_ETARIO"
    ]
])

,SEXO,NACIONALIDAD,PREVISION,TIPO_PROCEDENCIA,TIPO_INGRESO,IR_29301_PESO,IR_29301_SEVERIDAD,HOSPPROCEDENCIA,Nombre Región,COMUNA,RANGO_ETARIO
401706,MUJER,CHILE,FONASA D,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",PROGRAMADA,1.7114,0.0,NaN,Metropolitana de Santiago,ÑUÑOA,"(50, 60]"
464271,MUJER,CHILE,FONASA A,"APS URGENCIA (SAPU, SUR, SUC)",URGENCIA,1.7114,0.0,NaN,Metropolitana de Santiago,LA GRANJA,"(50, 60]"
480795,HOMBRE,CHILE,FONASA A,"APS URGENCIA (SAPU, SUR, SUC)",URGENCIA,1.7114,0.0,NaN,Metropolitana de Santiago,ÑUÑOA,"(50, 60]"
499763,MUJER,CHILE,FONASA B,"APS URGENCIA (SAPU, SUR, SUC)",URGENCIA,1.7114,0.0,NaN,Metropolitana de Santiago,ÑUÑOA,"(60, 70]"
499812,HOMBRE,CHILE,FONASA D,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",URGENCIA,1.7114,0.0,NaN,Metropolitana de Santiago,ÑUÑOA,"(70, 80]"
512641,HOMBRE,CHILE,FONASA B,OTROS HOSPITALES DE LA RED,URGENCIA,1.7114,0.0,"HOSPITAL DR. LUIS TISNÉ B. (SANTIAGO, PEÑALOLÉN)",Metropolitana de Santiago,PEÑALOLÉN,"(70, 80]"
513160,HOMBRE,ARGENTINA,NO IDENTIFICADA,OTROS HOSPITALES DE LA RED,URGENCIA,1.7114,0.0,"HOSPITAL DEL SALVADOR (SANTIAGO, PROVIDENCIA)",Metropolitana de Santiago,PROVIDENCIA,"(70, 80]"
551734,HOMBRE,CHILE,FONASA A,OTROS HOSPITALES DE LA RED,URGENCIA,1.7114,0.0,"HOSPITAL DEL SALVADOR (SANTIAGO, PROVIDENCIA)",Metropolitana de Santiago,LA CISTERNA,"(60, 70]"
649982,MUJER,CHILE,FONASA A,"APS URGENCIA (SAPU, SUR, SUC)",URGENCIA,1.7114,0.0,NaN,de Valparaíso,VIÑA DEL MAR,"(80, 90]"
789463,MUJER,CHILE,FONASA B,OTROS HOSPITALES DE LA RED,URGENCIA,1.7114,0.0,"HOSPITAL DEL SALVADOR (SANTIAGO, PROVIDENCIA)",Metropolitana de Santiago,PROVIDENCIA,"(70, 80]"


## 3. Análisis de Procedimientos 3

En este apartado se quieren identificar los rankings de procedimientos de alta relevancia/complejidad
(PROCEDIMIENTO1 hasta PROCEDIMIENTO3).

In [26]:
procedimientos_3_long = pd.melt(
    df_procesada,
    id_vars=[
        "COD_HOSPITAL",
        "ANIO_EGRESO",
        "TIPO_ACTIVIDAD",
        "CIP_ENCRIPTADO",
        "TIPOALTA",
        "DIAGNOSTICO1",
        "IR_29301_PESO",
        "IR_29301_SEVERIDAD",
    ],
    value_vars=["PROCEDIMIENTO1", "PROCEDIMIENTO2", "PROCEDIMIENTO3"],
    value_name="procedimiento",
)

In [27]:
var_categorica_a_contar = "procedimiento"
desglose_a_obtener = ["ANIO_EGRESO", "COD_HOSPITAL"]
grupo_ranking = ["ANIO_EGRESO", "procedimiento"]

conteo_y_ranking_procedimientos_3 = (
    build_features.calcular_conteo_y_ranking_sobre_variable_categorica(
        procedimientos_3_long,
        var_categorica_a_contar,
        desglose_a_obtener,
        grupo_ranking,
    )
)

In [28]:
conteo_y_ranking_procedimientos_3 = conteo_y_ranking_procedimientos_3.merge(
    cie_9, how="left", left_on="procedimiento", right_on="Código"
).merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)

In [29]:
conteo_y_ranking_procedimientos_3.to_excel("../data/interim/procedimientos_3_grd.xlsx", index=False)

## 4. Obtención de canastas de procedimientos por diagnóstico

En este análisis se quieren responder las siguientes preguntas:

- ¿Cuántos procedimientos se le deben realizar a una persona que tenga X diagnóstico 
(Ej: Cáncer de Pulmón)?
- ¿Cuál es el procedimiento más realizado dentro de un diagnóstico?
- ¿En qué diagnósticos es más utilizado X procedimiento (Ej: Angioplastia)?

Esta pregunta se quiere responder únicamente para el Hospital del Tórax. Además, para tener un
análisis promedio se utilizaran los años 2019, 2020 y 2021 en conjunto para responder las preguntas
anteriores. Además, se utilizarán todos los tipo de actividad realizada 
(HOSPITALIZACIÓN, HOSPITALIZACIÓN DIURNA, CMA, etc).

In [8]:
procedimientos_torax_long = procedimientos_formato_long.query("COD_HOSPITAL == @HOSPITAL_DEL_TORAX")

In [9]:
DIAGNOSTICOS_MAS_RELEVANTES = [
    "C33",
    "C34.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C38.1",
    "C38.4",
    "C45.0",
    "C78.0",
    "C78.2",
    "D14.3",
    "D38.1",
    "E84.8",
    "I05.1",
    "I08.0",
    "I08.1",
    "I34.0",
    "I35.0",
    "I35.1",
    "I35.2",
    "I42.0",
    "I45.6",
    "I47.2",
    "I49.5",
    "I71.0",
    "I71.2",
    "J39.8",
    "J47",
    "J67.9",
    "J84.1",
    "J84.8",
    "J84.9",
    "J86.0",
    "J86.9",
    "J90",
    "J93.1",
    "J95.5",
    "J98.0",
    "M34.8",
    "Q21.1",
    "Q23.1",
    "Q67.6",
    "T82.0",
    "T82.1",
    "Z45.0",
]

In [13]:
display(procedimientos_torax_long)

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,CIP_ENCRIPTADO,TIPOALTA,DIAGNOSTICO1,IR_29301_PESO,IR_29301_SEVERIDAD,variable,procedimiento
3946,112103,2019.0,HOSPITALIZACIÓN,1137216.0,DOMICILIO,I25.1,0.8838,1.0,PROCEDIMIENTO1,88.55
9385,112103,2019.0,HOSPITALIZACIÓN,1084360.0,DOMICILIO,Z51.1,1.9970,3.0,PROCEDIMIENTO1,99.25
9442,112103,2019.0,HOSPITALIZACIÓN,822388.0,DOMICILIO,Z51.1,0.9168,2.0,PROCEDIMIENTO1,99.25
9443,112103,2019.0,HOSPITALIZACIÓN DIURNA,317615.0,DERIVACIÓN OTRO HOSPITAL DEL SERVICIO,I21.3,1.7114,0.0,PROCEDIMIENTO1,00.66
9539,112103,2019.0,HOSPITALIZACIÓN,1397457.0,DOMICILIO,I35.0,3.2340,1.0,PROCEDIMIENTO1,35.21
...,...,...,...,...,...,...,...,...,...,...
82470182,112103,2021.0,HOSPITALIZACIÓN,75968176.0,DOMICILIO,U07.1,1.5219,3.0,PROCEDIMIENTO30,NaN
82470668,112103,2021.0,HOSPITALIZACIÓN,81034371.0,DOMICILIO,I05.9,7.7014,3.0,PROCEDIMIENTO30,NaN
82471078,112103,2021.0,HOSPITALIZACIÓN,67821076.0,DERIVACIÓN OTRO HOSPITAL DE LA RED NACIONAL,I21.9,2.4840,3.0,PROCEDIMIENTO30,NaN
82471166,112103,2021.0,HOSPITALIZACIÓN,76198942.0,DOMICILIO,I21.4,1.7184,1.0,PROCEDIMIENTO30,NaN


Se realizó el filtro de los procedimientos, solamente dejando los del hospital del Tórax.

Ahora, para obtener una cartera de procedimientos por diagnósticos se realizarán los siguientes
pasos (similar al realizado en el ámbito ambulatorio):

1. Se obtendrá el conteo de procedimientos por diagnóstico
2. Se obtendrá la cantidad de pacientes únicos por diagnóstico
3. Finalmente, se obtendrá la proporción de procedimientos por paciente para cada uno de los
diagnósticos.

In [15]:
canasta_procedimientos_hospitalizados = (
    (build_features.obtener_cartera_de_procedimientos_por_diagnostico(procedimientos_torax_long))
    .merge(cie_9, how="left", left_on="procedimiento", right_on="Código")
    .merge(cie_10, how="left", left_on="DIAGNOSTICO1", right_on="Código")
)

In [18]:
display(
    canasta_procedimientos_hospitalizados.query(
        "DIAGNOSTICO1 == 'C34.1' and procedimiento == '99.29'"
    )
)

,ANIO_EGRESO,DIAGNOSTICO1,procedimiento,cantidad_procedimientos,cantidad_pacientes_distintos,cantidad_proced_por_pacientes,Código_x,Descripción_x,Categoría_x,Sección_x,Capítulo_x,Versión,Código_y,Descripción_y,Categoría_y,Sección_y,Capítulo_y
977,2019.0,C34.1,99.29,175,176,0.994318,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
14337,2020.0,C34.1,99.29,41,70,0.585714,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)
23654,2021.0,C34.1,99.29,52,60,0.866667,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,99 OTROS PROCEDIMIENTOS NO QUIRURGICOS,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,CIE-v2013,C34.1,"Tumor maligno del lóbulo superior, bronquio o ...",C34 TUMOR MALIGNO DE LOS BRONQUIOS Y DEL PULMÓN,C30-C39 NEOPLASIAS MALIGNAS DE ÓRGANOS RESPIR...,Cap.02 NEOPLASIAS (C00-D49)


In [19]:
canasta_proceds_diags_mas_relevantes = canasta_procedimientos_hospitalizados[
    canasta_procedimientos_hospitalizados["DIAGNOSTICO1"].isin(DIAGNOSTICOS_MAS_RELEVANTES)
]

In [20]:
with pd.ExcelWriter("../data/interim/canasta_procedimientos_hospitalizados.xlsx") as file:
    canasta_procedimientos_hospitalizados.to_excel(
        file, sheet_name="canastas_globales", index=False
    )
    canasta_proceds_diags_mas_relevantes.to_excel(
        file, sheet_name="canasta_diags_mas_relevantes", index=False
    )

Los resultados muestran los procedimientos realizados para el diagnóstico C34.1.

- En primera lugar, se observa que el procedimiento más frecuente es el 90.59, realizado 283 veces. 
- Para el diagnóstico C34.1, existen 302 pacientes distintos entre 2019 y 2021.
- Finalmente, un paciente con un diagnóstico C34.1 se realiza en promedio 0.94 procedimientos 90.59.

Algo importante a tener en cuenta es que un mismo paciente se pudo haber realizado 10 veces
un procedimiento 90.59. Sin embargo, este análisis solamente da el promedio de procedimientos por
pacientes.

Algo interesante a responder sería saber cuál es la distribución en la cantidad de procedimientos
por diagnóstico. En otras palabras, responder las siguientes preguntas:

- ¿Cuántos procedimientos X se realiza un paciente que tiene el diagnóstico X? (Ya está respondido 
en el último apartado)
- **¿Cuántos es el máximo/mínimo de procedimientos X que se ha realizado un paciente que tiene el
diagnóstico X?**

A modo de control, se observará si el diagnóstico C34.1 efectivamente tiene 
283 procedimientos 90.59 y 302 pacientes distintos.

In [35]:
control_procedimientos = df_procesada.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX and DIAGNOSTICO1 == 'C34.1'"
)

display(control_procedimientos)

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA,RANGO_ETARIO,cod_comuna,codregion,Nombre Región
16208,112103,1129042.0,HOMBRE,1943-10-27,NINGUNO,SANTIAGO,ÑUÑOA,CHILE,FONASA B,METROPOLITANO ORIENTE,...,1.0,NaN,0.0,2019.0,2.0,2019-2,"(70, 80]",13120,13,Metropolitana de Santiago
16246,112103,1101478.0,HOMBRE,1961-04-12,NINGUNO,SANTIAGO,MAIPU,CHILE,FONASA D,METROPOLITANO CENTRAL,...,3.0,HOSPITAL CLÍNICO METROPOLITANO EL CARMEN DOCTO...,16.0,2019.0,3.0,2019-3,"(50, 60]",13119,13,Metropolitana de Santiago
16257,112103,281602.0,MUJER,1936-02-06,NINGUNO,SANTIAGO,ÑUÑOA,CHILE,FONASA B,METROPOLITANO ORIENTE,...,3.0,"HOSPITAL DEL SALVADOR (SANTIAGO, PROVIDENCIA)",1.0,2019.0,3.0,2019-3,"(80, 90]",13120,13,Metropolitana de Santiago
16262,112103,641101.0,MUJER,1945-03-05,NINGUNO,LINARES,LINARES,CHILE,FONASA B,DEL MAULE,...,3.0,NaN,11.0,2019.0,1.0,2019-1,"(70, 80]",7401,7,del Maule
18845,112103,1166456.0,HOMBRE,1950-05-02,NINGUNO,COIHAIQUE,COIHAIQUE,CHILE,FONASA B,AYSEN,...,1.0,HOSPITAL REGIONAL (COIHAIQUE),11.0,2019.0,6.0,2019-6,"(60, 70]",11101,11,de Aisén del Gral. C. Ibáñez del Campo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678748,112103,68618209.0,MUJER,1949-12-08,NINGUNO,CACHAPOAL,RANCAGUA,CHILE,FONASA B,LIBERTADOR B. O HIGGINS,...,1.0,NaN,7.0,2021.0,5.0,2021-5,"(70, 80]",6101,6,del Libertador B. O'Higgins
2680142,112103,84741484.0,HOMBRE,1950-12-18,NINGUNO,SANTIAGO,SAN MIGUEL,CHILE,FONASA D,METROPOLITANO SUR,...,1.0,NaN,5.0,2021.0,6.0,2021-6,"(70, 80]",13130,13,Metropolitana de Santiago
2690273,112103,69853991.0,MUJER,1944-10-26,NINGUNO,SANTIAGO,MACUL,CHILE,FONASA B,METROPOLITANO ORIENTE,...,2.0,NaN,4.0,2021.0,12.0,2021-12,"(70, 80]",13118,13,Metropolitana de Santiago
2694816,112103,67743046.0,MUJER,1939-09-03,NINGUNO,SANTIAGO,LAS CONDES,CHILE,FONASA B,METROPOLITANO ORIENTE,...,1.0,NaN,6.0,2021.0,3.0,2021-3,"(80, 90]",13114,13,Metropolitana de Santiago


En primer lugar, se observa que el diagnóstico C34.1 ha tenido 367 egresos entre 2019 y 2021.

In [36]:
control_pacientes_distintos = (control_procedimientos.CIP_ENCRIPTADO.unique())
print(f"Para el diagnostico C34.1 hay {len(control_pacientes_distintos)} pacientes distintos")

Para el diagnostico C34.1 hay 302 pacientes distintos


Por lo tanto, efectivamente existen 302 pacientes distintos para el diagnóstico C34.1.

Solamente queda saber si se realizaron 283 procedimientos 90.59.

In [37]:
control_suma_procedimientos = (control_procedimientos[vars_procedimientos] == "90.59").sum()
display(control_suma_procedimientos)
print(f"Para el diagnostico C34.1 hay {control_suma_procedimientos.sum()} procedimientos 90.59")

PROCEDIMIENTO1      2
PROCEDIMIENTO2      4
PROCEDIMIENTO3      8
PROCEDIMIENTO4      8
PROCEDIMIENTO5     15
PROCEDIMIENTO6     14
PROCEDIMIENTO7     14
PROCEDIMIENTO8     18
PROCEDIMIENTO9     16
PROCEDIMIENTO10    10
PROCEDIMIENTO11    17
PROCEDIMIENTO12    15
PROCEDIMIENTO13    14
PROCEDIMIENTO14    17
PROCEDIMIENTO15    15
PROCEDIMIENTO16    13
PROCEDIMIENTO17    16
PROCEDIMIENTO18     7
PROCEDIMIENTO19    10
PROCEDIMIENTO20     9
PROCEDIMIENTO21     6
PROCEDIMIENTO22    11
PROCEDIMIENTO23     5
PROCEDIMIENTO24     5
PROCEDIMIENTO25     2
PROCEDIMIENTO26     4
PROCEDIMIENTO27     6
PROCEDIMIENTO28     1
PROCEDIMIENTO29     0
PROCEDIMIENTO30     1
dtype: int64

Para el diagnostico C34.1 hay 283 procedimientos 90.59


Por lo tanto, efectivamente se realizaron 283 procedimiento 90.59.

**Por ende, el análisis de canasta de procedimientos se ha realizado correctamente.**